In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None, "display.max_columns", None)


import datetime
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
from keras.models import Sequential
from keras.layers import Dense
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load the dataset
ratings  = pd.read_csv('drive/MyDrive/employeeRetentionPrediction/data/ratings.csv')
remarks = pd.read_csv('drive/MyDrive/employeeRetentionPrediction/data/remarks.csv')
remarks_supp_opp = pd.read_csv('drive/MyDrive/employeeRetentionPrediction/data/remarks_supp_opp.csv')
test = pd.read_csv('drive/MyDrive/employeeRetentionPrediction/data/test.csv')
train = pd.read_csv('drive/MyDrive/employeeRetentionPrediction/data/train.csv')

In [4]:
# convert the date type columns to date
ratings['Date'] = pd.to_datetime(ratings['Date'])
remarks['remarkDate'] = pd.to_datetime(remarks['remarkDate'])
train['lastratingdate'] = pd.to_datetime(train['lastratingdate'])
test['lastratingdate'] = pd.to_datetime(test['lastratingdate'])

In [5]:
remarks.head()

,emp,comp,remarkId,txt,remarkDate
0,307,bnivzbfi,fvwadfrj,**********************************************...,2017-03-20
1,382,bnivzbfi,xrrfkgap,*****************************,2017-03-20
2,172,bnivzbfi,xkcrccwi,***************************,2017-03-20
3,135,bnivzbfi,lohhvtmo,***************************,2017-03-20
4,225,bnivzbfi,gpxxmoab,*********************************,2017-03-20


In [6]:
# Calculate age of remark taking the base as 2014. This might help the model to differentiate between the remarks of user
# For example, if it is an old remark by the employee then it will be given less weightage compared to a newer remark
years = np.array(remarks['remarkDate'].dt.year)
months = np.array(remarks['remarkDate'].dt.month)
days = np.array(remarks['remarkDate'].dt.day)
remarks['remarkAge']=(years-2014)+months/12+days/365

# convert the text of the remark to a number
# the give text column contains the remark in '********' format to hide the remarks. We will just take the its length.
remarks['txt_len'] = remarks['txt'].str.len()
remarks['txt_len_log'] = np.log10(np.array(list(remarks['txt_len'])))

# Make a new dataset from the remarks dataset by dropping the text, remarkDate and txt_len as all these were used to generate two new column 'remarkAge' and 'txt_len_log'
df_rem_edit = remarks.drop(['txt','remarkDate','txt_len'],axis=1)
df_rem_edit.head()

,emp,comp,remarkId,remarkAge,txt_len_log
0,307,bnivzbfi,fvwadfrj,3.304795,1.792392
1,382,bnivzbfi,xrrfkgap,3.304795,1.462398
2,172,bnivzbfi,xkcrccwi,3.304795,1.431364
3,135,bnivzbfi,lohhvtmo,3.304795,1.431364
4,225,bnivzbfi,gpxxmoab,3.304795,1.518514


In [7]:
# remark_weight is a data frame that stores reactions to each remark via remarkId

supportweight=1
opposeweight=5
label_encoder = preprocessing.LabelEncoder()

# Convert the support column True-False to 1-0 encoding
remarks_supp_opp['support']= label_encoder.fit_transform(remarks_supp_opp['support'])

# Convert the oppose column True-False to 1-0 encoding
remarks_supp_opp['oppose']= label_encoder.fit_transform(remarks_supp_opp['oppose'])

# For each remark ID , sum the number of support and oppose, then drop 'emp' column
remark_weight = remarks_supp_opp.groupby(['remarkId']).sum()

remarkIdDF = remark_weight
remark_weight = remark_weight.reset_index()

remark_weight = remark_weight.drop(['emp'], axis=1)

# Now we have a remark weight dataset showing the total support and oppose votes from all the employees.
# Add a new column that calculates the 'netfeeling' for a employee according to the weights for 'support' and 'oppose' that we decided.
remark_weight['netfeeling'] = remark_weight['support'] - opposeweight*remark_weight['oppose']

# Add a column that calculates the total number of votes that each remark got
remark_weight['noofreactions'] = remark_weight['support'] + remark_weight['oppose']

# Add a new column to know the support weightage of each remark. For example, if a remark gets just support, the support_weightage will be 1.
remark_weight['support_weightage_for_a_remarkID'] = remark_weight['support']/( remark_weight['support'] + opposeweight*remark_weight['oppose'])

remark_weight.head()

,remarkId,support,oppose,netfeeling,noofreactions,support_weightage_for_a_remarkID
0,aaaayzmf,2,0,2,2,1.0
1,aaaixxee,7,0,7,7,1.0
2,aabtpvhw,3,0,3,3,1.0
3,aabtzbxo,8,0,8,8,1.0
4,aacsadyv,2,0,2,2,1.0


In [8]:
# Combine the modified version of the remarks dataset with the remark_weight dataset
df_rem = pd.merge(df_rem_edit,remark_weight,how='left',left_on='remarkId',right_on='remarkId')
df_rem = df_rem.drop(['remarkId'],axis=1)
df_rem.head()

,emp,comp,remarkAge,txt_len_log,support,oppose,netfeeling,noofreactions,support_weightage_for_a_remarkID
0,307,bnivzbfi,3.304795,1.792392,4.0,0.0,4.0,4.0,1.000000
1,382,bnivzbfi,3.304795,1.462398,1.0,2.0,-9.0,3.0,0.090909
2,172,bnivzbfi,3.304795,1.431364,3.0,0.0,3.0,3.0,1.000000
3,135,bnivzbfi,3.304795,1.431364,1.0,1.0,-4.0,2.0,0.166667
4,225,bnivzbfi,3.304795,1.518514,3.0,2.0,-7.0,5.0,0.230769


In [9]:
# Here I will try to get the latest 15 remark infor for each employee, company combination
df_full = pd.DataFrame(columns = {'emp', 'comp', 'age1', 'age2', 'age3', 'age4', 'age5', 'age6', 'age7', 'age8', 'age9', 'age10', 'age11', 'age12', 'age13', 'age14', 'age15',	
                               'len1', 'len2', 'len3', 'len4', 'len5', 'len6', 'len7', 'len8', 'len9', 'len10', 'len11', 'len12', 'len13', 'len14', 'len15',	
                               'support1', 'support2', 'support3', 'support4', 'support5', 'support6', 'support7', 'support8', 'support9', 'support10', 'support11', 'support12', 'support13', 'support14', 'support15',	
                               'oppose1', 'oppose2', 'oppose3', 'oppose4', 'oppose5', 'oppose6', 'oppose7', 'oppose8', 'oppose9', 'oppose10', 'oppose11', 'oppose12', 'oppose13', 'oppose14', 'oppose15',	
                               'netfeel1', 'netfeel2', 'netfeel3', 'netfeel4', 'netfeel5', 'netfeel6', 'netfeel7', 'netfeel8', 'netfeel9', 'netfeel10', 'netfeel11', 'netfeel12', 'netfeel13', 'netfeel14', 'netfeel15',	
                               'remarkCount1', 'remarkCount2', 'remarkCount3', 'remarkCount4', 'remarkCount5', 'remarkCount6', 'remarkCount7', 'remarkCount8', 'remarkCount9', 'remarkCount10', 'remarkCount11', 'remarkCount12', 'remarkCount13', 'remarkCount14', 'remarkCount15',	
                               'supprate1', 'supprate2', 'supprate3', 'supprate4', 'supprate5', 'supprate6', 'supprate7', 'supprate8', 'supprate9', 'supprate10', 'supprate11', 'supprate12', 'supprate13', 'supprate14', 'supprate15'})
data_index = 0
for (emp, comp),group in df_rem.groupby(['emp', 'comp']):
  new_row = {'emp': emp, 'comp': comp, 'age1':-1, 'age2':-1, 'age3':-1, 'age4':-1, 'age5':-1, 'age6':-1, 'age7':-1, 'age8':-1, 'age9':-1, 'age10':-1, 'age11':-1, 'age12':-1, 'age13':-1, 'age14':-1, 'age15':-1,	
            'len1':-1, 'len2':-1, 'len3':-1, 'len4':-1, 'len5':-1, 'len6':-1, 'len7':-1, 'len8':-1, 'len9':-1, 'len10':-1, 'len11':-1, 'len12':-1, 'len13':-1, 'len14':-1, 'len15':-1,	
            'support1':-1, 'support2':-1, 'support3':-1, 'support4':-1, 'support5':-1, 'support6':-1, 'support7':-1, 'support8':-1, 'support9':-1, 'support10':-1, 'support11':-1, 'support12':-1, 'support13':-1, 'support14':-1, 'support15':-1,	
            'oppose1':-1, 'oppose2':-1, 'oppose3':-1, 'oppose4':-1, 'oppose5':-1, 'oppose6':-1, 'oppose7':-1, 'oppose8':-1, 'oppose9':-1, 'oppose10':-1, 'oppose11':-1, 'oppose12':-1, 'oppose13':-1, 'oppose14':-1, 'oppose15':-1,	
            'netfeel1':-1, 'netfeel2':-1, 'netfeel3':-1, 'netfeel4':-1, 'netfeel5':-1, 'netfeel6':-1, 'netfeel7':-1, 'netfeel8':-1, 'netfeel9':-1, 'netfeel10':-1, 'netfeel11':-1, 'netfeel12':-1, 'netfeel13':-1, 'netfeel14':-1, 'netfeel15':-1,	
            'remarkCount1':-1, 'remarkCount2':-1, 'remarkCount3':-1, 'remarkCount4':-1, 'remarkCount5':-1, 'remarkCount6':-1, 'remarkCount7':-1, 'remarkCount8':-1, 'remarkCount9':-1, 'remarkCount10':-1, 'remarkCount11':-1, 'remarkCount12':-1, 'remarkCount13':-1, 'remarkCount14':-1, 'remarkCount15':-1,	
            'supprate1':-1, 'supprate2':-1, 'supprate3':-1, 'supprate4':-1, 'supprate5':-1, 'supprate6':-1, 'supprate7':-1, 'supprate8':-1, 'supprate9':-1, 'supprate10':-1, 'supprate11':-1, 'supprate12':-1, 'supprate13':-1, 'supprate14':-1, 'supprate15':-1}
  #data = data.append(new_row, ignore_index = True)

  df_full = df_full.append(pd.Series(new_row, name=data_index))
  

  group = group.reset_index()
  for ind in group.index:
    if(ind >= 15):
      break
    df_full.at[data_index, 'age'+str(ind+1)] = group['remarkAge'][ind]
    df_full.at[data_index, 'len'+str(ind+1)] = group['txt_len_log'][ind]
    df_full.at[data_index, 'support'+str(ind+1)] = group['support'][ind]
    df_full.at[data_index, 'oppose'+str(ind+1)] = group['oppose'][ind]
    df_full.at[data_index, 'netfeel'+str(ind+1)] = group['netfeeling'][ind]
    df_full.at[data_index, 'remarkCount'+str(ind+1)] = group['noofreactions'][ind]
    df_full.at[data_index, 'supprate'+str(ind+1)] = group['support_weightage_for_a_remarkID'][ind]

  data_index = data_index + 1


In [10]:
df_full = df_full[['emp', 'comp', 'age1', 'age2', 'age3', 'age4', 'age5', 'age6', 'age7', 'age8', 'age9', 'age10', 'age11', 'age12', 'age13', 'age14', 'age15',	
                               'len1', 'len2', 'len3', 'len4', 'len5', 'len6', 'len7', 'len8', 'len9', 'len10', 'len11', 'len12', 'len13', 'len14', 'len15',	
                               'support1', 'support2', 'support3', 'support4', 'support5', 'support6', 'support7', 'support8', 'support9', 'support10', 'support11', 'support12', 'support13', 'support14', 'support15',	
                               'oppose1', 'oppose2', 'oppose3', 'oppose4', 'oppose5', 'oppose6', 'oppose7', 'oppose8', 'oppose9', 'oppose10', 'oppose11', 'oppose12', 'oppose13', 'oppose14', 'oppose15',	
                               'netfeel1', 'netfeel2', 'netfeel3', 'netfeel4', 'netfeel5', 'netfeel6', 'netfeel7', 'netfeel8', 'netfeel9', 'netfeel10', 'netfeel11', 'netfeel12', 'netfeel13', 'netfeel14', 'netfeel15',	
                               'remarkCount1', 'remarkCount2', 'remarkCount3', 'remarkCount4', 'remarkCount5', 'remarkCount6', 'remarkCount7', 'remarkCount8', 'remarkCount9', 'remarkCount10', 'remarkCount11', 'remarkCount12', 'remarkCount13', 'remarkCount14', 'remarkCount15',	
                               'supprate1', 'supprate2', 'supprate3', 'supprate4', 'supprate5', 'supprate6', 'supprate7', 'supprate8', 'supprate9', 'supprate10', 'supprate11', 'supprate12', 'supprate13', 'supprate14', 'supprate15']]
df_full.head(25)

,emp,comp,age1,age2,age3,age4,age5,age6,age7,age8,age9,age10,age11,age12,age13,age14,age15,len1,len2,len3,len4,len5,len6,len7,len8,len9,len10,len11,len12,len13,len14,len15,support1,support2,support3,support4,support5,support6,support7,support8,support9,support10,support11,support12,support13,support14,support15,oppose1,oppose2,oppose3,oppose4,oppose5,oppose6,oppose7,oppose8,oppose9,oppose10,oppose11,oppose12,oppose13,oppose14,oppose15,netfeel1,netfeel2,netfeel3,netfeel4,netfeel5,netfeel6,netfeel7,netfeel8,netfeel9,netfeel10,netfeel11,netfeel12,netfeel13,netfeel14,netfeel15,remarkCount1,remarkCount2,remarkCount3,remarkCount4,remarkCount5,remarkCount6,remarkCount7,remarkCount8,remarkCount9,remarkCount10,remarkCount11,remarkCount12,remarkCount13,remarkCount14,remarkCount15,supprate1,supprate2,supprate3,supprate4,supprate5,supprate6,supprate7,supprate8,supprate9,supprate10,supprate11,supprate12,supprate13,supprate14,supprate15
0,-217,phcvroct,1.2379,1.2379,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2.41162,2.41162,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8.0,8.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8.0,8.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8.0,8.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-216,phcvroct,1.258219,1.240639,1.23516,1.258219,1.240639,1.23516,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.146128,1.591065,1.342423,1.146128,1.591065,1.342423,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,6.0,5.0,1.0,6.0,5.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,6.0,5.0,1.0,6.0,5.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,6.0,5.0,1.0,6.0,5.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-214,phcvroct,1.508219,1.508219,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.60206,0.60206,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-213,phcvroct,1.758219,1.758219,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.70757,1.70757,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-212,phcvroct,1.841553,1.841553,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.778151,0.778151,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-211,phcvroct,1.924886,1.924886,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2.117271,2.117271,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3.0,3.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3.0,3.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3.0,3.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,-210,phcvroct,1.285616,1.285616,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.90309,0.90309,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6.0,6.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6.0,6.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6.0,6.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,-209,phcvroct,1.302055,1.299315,1.296575,1.293836,1.285616,1.302055,1.299315,1.296575,1.293836,1.285616,-1,-1,-1,-1,-1,1.724276,1.176091,1.146128,1.176091,1.255273,1.724276,1.176091,1.146128,1.176091,1.255273,-1,-1,-1,-1,-1,1.0,1.0,3.0,NaN,NaN,1.0,1.0,3.0,NaN,NaN,-1,-1,-1,-1,-1,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,-1,-1,-1,-1,-1,1.0,1.0,3.0,NaN,NaN,1.0,1.0,3.0,NaN,NaN,-1,-1,-1,-1,-1,1.0,1.0,3.0,NaN,NaN,1.0,1.0,3.0,NaN,NaN,-1,-1,-1,-1,-1,1.0,1.0,

In [11]:
# Employee rating calculation
# here we are trying to determine the support/oppose the employee got for his remark from other employees    
group_by_emp = remarks.groupby(['emp'])

emp_rating=[]
empId=[]
for emp, grp in group_by_emp:
    lik = 0
    dis = 0
    # get the list of all the remarks given by an employee
    remarks_by_emp = list(grp['remarkId'])
    for c in remarks_by_emp:
        # Check the number of support votes an employee got for his/her remark from other employees
        try:
            l=remarkIdDF.loc[c].at['support']
        except:
            l=0
        # Check the number of oppose votes an employee got for his/her remark from other employees
        try:
            d=remarkIdDF.loc[c].at['oppose']
        except:
            d=0
        
        # add the total support/oppose votes an employee got for his/her remark
        lik += l
        dis += d
    empId.append(emp)
    # calculate the weighted fraction of support votes the employee got
    emp_rating.append(lik/(lik+opposeweight*dis) if (lik+opposeweight*dis)!=0 else 0)

In [12]:
# Make a dataframe with a column denoting the employee rating for each employee
# if an employee gets all support votes and no oppose votes, he/she will have a fraction of 1
# for a dislike, the fraction will reduce (drastically if opposeweight is very high)
# this might help to weigh an employee's remark.
df_employee_rating = pd.DataFrame({'emp':empId,'emp_rating':emp_rating})
df_employee_rating.head()

,emp,emp_rating
0,-217,1.0
1,-216,1.0
2,-214,0.0
3,-213,1.0
4,-212,0.0


In [13]:
# Create a Dataframe showing for each employee with his/her overall support fraction and value of his/her remark.

grp_by_emp = remarks_supp_opp.groupby(['emp'])

emp_remark_val=[]
emp_support_overall =[]
employee=[]
i = 0
for emp, grp in grp_by_emp:
  # calculate how many support votes an employee got in total for all his/her remarks
  supporting_rate = list(grp['support']).count(1)/len(grp['support'])
  # This is like IDF, if an employee has given a lot of remarks then the value will be close to zero
  # if an employee has given less remarks, the value will be large
  emp_rv = np.log10(len(remarks)/len(grp['support']))
  employee.append(emp)
  emp_remark_val.append(emp_rv)
  emp_support_overall.append(supporting_rate)

df_employee_support_rate = pd.DataFrame({'emp':employee,'emp_support_overall':emp_support_overall,'emp_remark_valueIDF':emp_remark_val})
df_employee_support_rate.head()

,emp,emp_support_overall,emp_remark_valueIDF
0,-218,1.0,4.917799
1,-216,1.0,4.218829
2,-215,1.0,4.917799
3,-211,1.0,3.519859
4,-210,1.0,4.616769


In [14]:
# Merge the employee_rating and employee_support_rate dataframe
emp_df=pd.merge(df_employee_rating,df_employee_support_rate,how='left',left_on='emp',right_on='emp')
emp_df = emp_df.dropna(axis=0)
emp_df.head()

,emp,emp_rating,emp_support_overall,emp_remark_valueIDF
1,-216,1.0,1.0,4.218829
5,-211,1.0,1.0,3.519859
6,-210,1.0,1.0,4.616769
7,-209,1.0,1.0,3.963557
8,-208,1.0,1.0,4.072701


In [15]:
# calculating avg rating by employee for companies 
empl=[]
cmpny=[]
avgrat=[]
group_by_empcomp = ratings.groupby(['emp','comp'])
for (emp,comp),grp in group_by_empcomp:
    empl.append(emp)
    cmpny.append(comp)
    # calculate the average of all the ratings an employee gave for a particular company
    avgrating = np.array(list(grp['rating'])).sum()/len(grp['rating'])
    avgrat.append(avgrating)
    
df_avgemp_rating_company = pd.DataFrame({'emp':empl,'comp':cmpny,'avg_emp_rat_comp':avgrat})    
df_avgemp_rating_company.head()

,emp,comp,avg_emp_rat_comp
0,1,azalutpt,4.000000
1,1,ejeyobsm,2.917127
2,1,ewpvmfbc,3.000000
3,1,fqsozvpv,2.000000
4,1,iqdwmigj,2.464286


In [16]:
# Calculate the avg rating each company
group_by_company= ratings.groupby(['comp'])
cmpny=[]
avgrat=[]
for comp,grp in group_by_company:
    cmpny.append(comp)
    # Here calculating the average rating that each company got
    avgrating = np.array(list(grp['rating'])).sum()/len(grp['rating'])
    avgrat.append(avgrating)
    
df_avg_comprating = pd.DataFrame({'comp':cmpny,'avg_rat_comp':avgrat})    
df_avg_comprating.head()

,comp,avg_rat_comp
0,azalutpt,3.456522
1,bhqczwkj,3.089277
2,bnivzbfi,3.236470
3,bucyzegb,3.396226
4,dmgwoqhz,3.475316


In [17]:
# Merge the above two generated datasets
df3 = pd.merge(df_avg_comprating,df_avgemp_rating_company,how='left',left_on='comp',right_on='comp')

# Calculate a new value 'satisfaction' which will be the subtraction of an employee's average rating given to a company - the company's average rating
df3['satisfaction']= df3['avg_emp_rat_comp']-df3['avg_rat_comp']
cols = df3.columns.tolist()
cols = [cols[2],cols[0],cols[3],cols[1],cols[4]]
df3 = df3[cols]
df3.head()

,emp,comp,avg_emp_rat_comp,avg_rat_comp,satisfaction
0,1,azalutpt,4.000000,3.456522,0.543478
1,3,azalutpt,3.500000,3.456522,0.043478
2,4,azalutpt,3.000000,3.456522,-0.456522
3,6,azalutpt,2.777778,3.456522,-0.678744
4,7,azalutpt,3.818182,3.456522,0.361660


In [18]:
df4 = df3.drop(['comp','avg_rat_comp'],axis=1) 
gbe = df4.groupby(['emp'])

e=[]
avgerat=[]
avgsat=[]
# For each employee get the average of average rating he/she has given to each company
# For each employee get the average of the satisfaction value 
for emp,grp in gbe:
    e.append(emp)
    avgerat.append(np.array(list(grp['avg_emp_rat_comp'])).sum()/len(list(grp['avg_emp_rat_comp'])))
    avgsat.append(np.array(list(grp['satisfaction'])).sum()/len(list(grp['satisfaction'])))
    
df_emp_sat_avg=pd.DataFrame({'emp':e,'avg_emp_rat_comp':avgerat,'satisfaction':avgsat})  
df_emp_sat_avg.head()

,emp,avg_emp_rat_comp,satisfaction
0,1,3.263681,0.298027
1,2,2.810497,-0.150796
2,3,3.276562,0.216704
3,4,2.859291,-0.113851
4,5,2.923655,-0.050811


In [19]:
# Merge the two dataframes
emp_overall = pd.merge(df_emp_sat_avg,emp_df,how='left', left_on='emp', right_on='emp')
emp_overall = emp_overall.fillna(0)
emp_overall.head()

,emp,avg_emp_rat_comp,satisfaction,emp_rating,emp_support_overall,emp_remark_valueIDF
0,1,3.263681,0.298027,0.447391,0.892394,1.553436
1,2,2.810497,-0.150796,0.463454,0.763618,1.628657
2,3,3.276562,0.216704,0.560581,0.843373,1.676502
3,4,2.859291,-0.113851,0.535458,0.674738,1.733676
4,5,2.923655,-0.050811,0.563107,0.836512,2.052103


In [20]:
# merge with the main dataset
emp_data = pd.merge(df_full,emp_overall,how='left', left_on='emp', right_on='emp')
emp_data.tail()

,emp,comp,age1,age2,age3,age4,age5,age6,age7,age8,age9,age10,age11,age12,age13,age14,age15,len1,len2,len3,len4,len5,len6,len7,len8,len9,len10,len11,len12,len13,len14,len15,support1,support2,support3,support4,support5,support6,support7,support8,support9,support10,support11,support12,support13,support14,support15,oppose1,oppose2,oppose3,oppose4,oppose5,oppose6,oppose7,oppose8,oppose9,oppose10,oppose11,oppose12,oppose13,oppose14,oppose15,netfeel1,netfeel2,netfeel3,netfeel4,netfeel5,netfeel6,netfeel7,netfeel8,netfeel9,netfeel10,netfeel11,netfeel12,netfeel13,netfeel14,netfeel15,remarkCount1,remarkCount2,remarkCount3,remarkCount4,remarkCount5,remarkCount6,remarkCount7,remarkCount8,remarkCount9,remarkCount10,remarkCount11,remarkCount12,remarkCount13,remarkCount14,remarkCount15,supprate1,supprate2,supprate3,supprate4,supprate5,supprate6,supprate7,supprate8,supprate9,supprate10,supprate11,supprate12,supprate13,supprate14,supprate15,avg_emp_rat_comp,satisfaction,emp_rating,emp_support_overall,emp_remark_valueIDF
3214,968,phcvroct,2.696804,2.888128,2.610731,2.826712,2.810274,2.793836,2.788356,2.785616,2.696804,2.888128,2.610731,2.826712,2.810274,2.793836,2.788356,0.90309,1.20412,1.361728,1.380211,1.531479,1.0,1.748188,1.380211,0.90309,1.20412,1.361728,1.380211,1.531479,1.0,1.748188,1.0,5.0,4.0,10.0,6.0,4.0,11.0,6.0,1.0,5.0,4.0,10.0,6.0,4.0,11.0,4.0,0.0,3.0,1.0,0.0,2.0,0.0,4.0,4.0,0.0,3.0,1.0,0.0,2.0,0.0,-19.0,5.0,-11.0,5.0,6.0,-6.0,11.0,-14.0,-19.0,5.0,-11.0,5.0,6.0,-6.0,11.0,5.0,5.0,7.0,11.0,6.0,6.0,11.0,10.0,5.0,5.0,7.0,11.0,6.0,6.0,11.0,0.047619,1.0,0.210526,0.666667,1.0,0.285714,1.0,0.230769,0.047619,1.0,0.210526,0.666667,1.0,0.285714,1.0,2.000000,-0.591810,0.401709,1.000000,3.687351
3215,969,phcvroct,2.885388,2.860731,2.885388,2.860731,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.20412,1.70757,1.20412,1.70757,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2.0,9.0,2.0,9.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,3.0,0.0,3.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2.0,-6.0,2.0,-6.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2.0,12.0,2.0,12.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1.0,0.375,1.0,0.375,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3.222222,0.630412,0.423077,0.857143,3.771671
3216,970,phcvroct,3.505479,3.088813,3.165525,3.157306,3.154566,3.151826,2.901826,2.890868,2.888128,2.882648,2.610731,2.444064,2.832192,2.823973,2.785616,1.0,2.117271,2.49276,1.799341,2.38739,1.568202,1.94939,2.786041,2.700704,1.755875,1.447158,1.977724,2.056905,2.87564,2.123852,4.0,12.0,43.0,18.0,36.0,20.0,3.0,33.0,28.0,2.0,7.0,7.0,9.0,38.0,9.0,7.0,0.0,1.0,3.0,1.0,0.0,2.0,0.0,3.0,4.0,2.0,1.0,1.0,0.0,0.0,-31.0,12.0,38.0,3.0,31.0,20.0,-7.0,33.0,13.0,-18.0,-3.0,2.0,4.0,38.0,9.0,11.0,12.0,44.0,21.0,37.0,20.0,5.0,33.0,31.0,6.0,9.0,8.0,10.0,38.0,9.0,0.102564,1.0,0.895833,0.545455,0.878049,1.0,0.230769,1.0,0.651163,0.090909,0.411765,0.583333,0.642857,1.0,1.0,2.416667,-0.175144,0.682741,0.945813,2.610303
3217,996,phcvroct,3.038356,3.035616,2.813014,2.785616,2.607991,2.441324,3.038356,3.035616,2.813014,2.785616,2.607991,2.441324,-1,-1,-1,1.991226,1.986772,2.30103,1.176091,1.939519,0.69897,1.991226,1.986772,2.30103,1.176091,1.939519,0.69897,-1,-1,-1,4.0,6.0,23.0,5.0,4.0,0.0,4.0,6.0,23.0,5.0,4.0,0.0,-1,-1,-1,1.0,0.0,1.0,0.0,2.0,2.0,1.0,0.0,1.0,0.0,2.0,2.0,-1,-1,-1,-1.0,6.0,18.0,5.0,-6.0,-10.0,-1.0,6.0,18.0,5.0,-6.0,-10.0,-1,-1,-1,5.0,6.0,24.0,5.0,6.0,2.0,5.0,6.0,24.0,5.0,6.0,2.0,-1,-1,-1,0.444444,1.0,0.821429,1.0,0.285714,0.0,0.444444,1.0,0.821429,1.0,0.285714,0.0,-1,-1,-1,2.450000,-0.141810,0.583333,0.909091,3.575377
3218,999,phcvroct,2.860731,2.610731,2.829452,2.823973,2.860731,2.610731,2.829452,2.823973,-1,-1,-1,-1,-1,-1,-1,2.264818,2.079181,2.214844,1.778151,2.264818,2.079181,2.214844,1.778151,-1,-1,-1,-1,-1,-1,-1,4.0,3.0,6.0,5.0,4.0,3.0,6.0,5.0,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,4.0,1.0,0.0,0.0,4.0,1.0,-1,-1,-1,-1,-1,-1,-1,4.0,3.0,-14.0,0.0,4.0,3.0,-14.0,0.0,-1,-1,-1,-1,-1,-1,-1,4.0,3.0,10.0,6.0,4.0,3.0,10.0,6.0,-1,-1,-1,-1,-1,-1,-1,1.0,1.0,0.230769,0.5,1.0,1.0,0.230769,0.5,-1,-1,-1,-1,-1,-1,-1,2.850000,0.258190,0.418605,0.5

In [21]:
# calculate avg rating for each company

# Group by company
group_by_company= ratings.groupby(['comp'])

cmpny=[]
avgrat=[]
# For each company, calculate the average rating given by all employee
for comp,grp in group_by_company:
    cmpny.append(comp)
    avgrating = np.array(list(grp['rating'])).sum()/len(grp['rating'])
    avgrat.append(avgrating)

# A dataframe which gives the average rating for each company    
df_avg_comprating = pd.DataFrame({'comp':cmpny,'avg_rat_comp':avgrat})    

df_avg_comprating.head()

,comp,avg_rat_comp
0,azalutpt,3.456522
1,bhqczwkj,3.089277
2,bnivzbfi,3.236470
3,bucyzegb,3.396226
4,dmgwoqhz,3.475316


In [22]:
grp_emp_comp = remarks_supp_opp.groupby(['emp','comp'])
remarks_supported_for_company =[]
company=[]
employee =[]
# Here we will try to calculate the fraction of supporting remarks an employee gave to a company
for (emp,comp) , grp in grp_emp_comp:
    # calculate The fraction of supporting vote given by an employee to a company
    remarks_supported_for_company.append(list(grp['support']).count(1)/len(grp['support']))
    employee.append(emp)
    company.append(comp)
df_employee_for_company = pd.DataFrame({'emp':employee,'comp':company,'remarks_supported_for_a_company_by_an_employee':remarks_supported_for_company})
df_employee_for_company.head()

,emp,comp,remarks_supported_for_a_company_by_an_employee
0,-218,phcvroct,1.0
1,-216,phcvroct,1.0
2,-215,phcvroct,1.0
3,-211,phcvroct,1.0
4,-210,phcvroct,1.0


In [23]:
## Here we will try to get some features from ratings dataset
group_emp = ratings.groupby(['emp','comp'])

employee = []
company = []
avg_rating_weighted_arr = []
no_of_ratings_for_a_comp_by_an_emp = []
rating_duration = []
rating_variance_arr = []

for (emp, comp), group in group_emp:
    group = group.reset_index().sort_values(by=['Date'])
    group = group.reset_index()

    # We will give more importance to the latest ratings.
    weights =[i**2 for i in range(len(group))]

    # Multiply the weights with the 'ratings' column also divide by total sum of weights
    avg_rating_weighted = (group['rating']*weights).sum()/(sum(weights))
    
    employee.append(emp)
    company.append(comp)
    # create feature for weighted rating
    avg_rating_weighted_arr.append(avg_rating_weighted)
    # create a feature for no of ratings an employee gives to a company
    no_of_ratings_for_a_comp_by_an_emp.append(len(group))
    # create feature for total duration of rating that an employee has given to a company
    rating_duration.append((group['Date'][len(group)-1] - group['Date'][0]).days / 365)
    # create a feature for variance of rating
    rating_variance_arr.append(group['rating'].std())
    
df_rating_statistics = pd.DataFrame({'emp':employee,'comp':company,'avg_rating_weighted':avg_rating_weighted_arr,'no_of_ratings_for_a_comp_by_an_emp':no_of_ratings_for_a_comp_by_an_emp,'rating_duration':rating_duration,'rating_variance':rating_variance_arr})
df_rating_statistics.head()

,emp,comp,avg_rating_weighted,no_of_ratings_for_a_comp_by_an_emp,rating_duration,rating_variance
0,1,azalutpt,4.000000,2,0.002740,0.000000
1,1,ejeyobsm,2.914823,181,1.904110,0.556561
2,1,ewpvmfbc,NaN,1,0.000000,NaN
3,1,fqsozvpv,NaN,1,0.000000,NaN
4,1,iqdwmigj,2.474061,56,1.723288,0.830428


In [24]:
train.head()

,id,emp,comp,lastratingdate,left
0,2228,939,phcvroct,2016-10-17,1
1,4349,250,jblrepyr,2017-03-19,0
2,945,134,ewpvmfbc,2016-09-21,0
3,4553,164,wsmblohy,2017-03-17,0
4,941,129,ewpvmfbc,2016-04-04,0


In [25]:
### Important feature
# calculate the fraction of employees leaving a company
grp_dfs = train.groupby(['comp'])
list_comp=[]
left_percent = []
for comp, grp in grp_dfs:
    list_comp.append(comp)
    # Calculate the fraction of employees leaving a particular company
    left_percent.append(list(grp['left']).count(1)/len(list(grp['left'])))
comp_info_df = pd.DataFrame({'comp':list_comp,'company_leaving_percent_all':left_percent})

In [26]:
#### important feature
# probability of employee leaving for all companies
grp_dfs = train.groupby(['emp'])
employee_leaving_probability =[]
list_emp=[]
for emp, grp in grp_dfs:
    list_emp.append(emp)
    # The probability that an employee will leave any company
    employee_leaving_probability.append(list(grp['left']).count(1)/len(list(grp['left'])))
emp_leaving_info_df = pd.DataFrame({'emp':list_emp,'employee_leaving_percent':employee_leaving_probability})

In [27]:
# Merge all the features that we have extracted from the analysis above

train_final_df = pd.merge(left=train, right=comp_info_df, how='left')
test_final_df = pd.merge(left=test, right=comp_info_df, how='left')

train_final_df = pd.merge(left=train_final_df, right=emp_leaving_info_df, how='left')
test_final_df = pd.merge(left=test_final_df, right=emp_leaving_info_df, how='left')

train_final_df = pd.merge(left=train_final_df, right=emp_data, how='left')
test_final_df = pd.merge(left=test_final_df, right=emp_data, how='left')

train_final_df = pd.merge(left=train_final_df, right=df_avg_comprating, how='left')
test_final_df = pd.merge(left=test_final_df, right=df_avg_comprating, how='left')

train_final_df = pd.merge(left=train_final_df, right=df_employee_for_company, how='left')
test_final_df = pd.merge(left=test_final_df, right=df_employee_for_company, how='left')

train_final_df = pd.merge(left=train_final_df, right=df_rating_statistics, how='left')
test_final_df = pd.merge(left=test_final_df, right=df_rating_statistics, how='left')

train_final_df = pd.get_dummies(train_final_df, columns = ['comp'],prefix = ['comp'])
test_final_df = pd.get_dummies(test_final_df, columns = ['comp'],prefix = ['comp'])

train_final_df = train_final_df.drop(['lastratingdate'], axis=1)
test_final_df = test_final_df.drop(['lastratingdate'], axis=1)

id_df = test_final_df['id']
train_final_df = train_final_df.drop(['id'], axis=1)
test_final_df = test_final_df.drop(['id'], axis=1)

test_final_df['comp_azalutpt'] = 0
test_final_df['comp_fjslutlg'] = 0

target = train_final_df['left']
train_final_df = train_final_df.drop(['left'], axis=1)

test_final_df = test_final_df[train_final_df.columns.tolist()]

train_final_df = train_final_df.fillna(-1)
test_final_df = test_final_df.fillna(-1)

In [28]:
train_final_df.columns

Index(['emp', 'company_leaving_percent_all', 'employee_leaving_percent',
       'age1', 'age2', 'age3', 'age4', 'age5', 'age6', 'age7',
       ...
       'comp_ugldwwzf', 'comp_ujplihug', 'comp_vcqsbirc', 'comp_vwcdylha',
       'comp_wsmblohy', 'comp_xccmgbjz', 'comp_ydqdpmvi', 'comp_ylpksopb',
       'comp_yodaczsb', 'comp_zptfoxyq'],
      dtype='object', length=156)

In [29]:
train_final_df = train_final_df.drop(['emp'], axis=1)
test_final_df = test_final_df.drop(['emp'], axis=1)

In [30]:
train_final_df.head()

,company_leaving_percent_all,employee_leaving_percent,age1,age2,age3,age4,age5,age6,age7,age8,age9,age10,age11,age12,age13,age14,age15,len1,len2,len3,len4,len5,len6,len7,len8,len9,len10,len11,len12,len13,len14,len15,support1,support2,support3,support4,support5,support6,support7,support8,support9,support10,support11,support12,support13,support14,support15,oppose1,oppose2,oppose3,oppose4,oppose5,oppose6,oppose7,oppose8,oppose9,oppose10,oppose11,oppose12,oppose13,oppose14,oppose15,netfeel1,netfeel2,netfeel3,netfeel4,netfeel5,netfeel6,netfeel7,netfeel8,netfeel9,netfeel10,netfeel11,netfeel12,netfeel13,netfeel14,netfeel15,remarkCount1,remarkCount2,remarkCount3,remarkCount4,remarkCount5,remarkCount6,remarkCount7,remarkCount8,remarkCount9,remarkCount10,remarkCount11,remarkCount12,remarkCount13,remarkCount14,remarkCount15,supprate1,supprate2,supprate3,supprate4,supprate5,supprate6,supprate7,supprate8,supprate9,supprate10,supprate11,supprate12,supprate13,supprate14,supprate15,avg_emp_rat_comp,satisfaction,emp_rating,emp_support_overall,emp_remark_valueIDF,avg_rat_comp,remarks_supported_for_a_company_by_an_employee,avg_rating_weighted,no_of_ratings_for_a_comp_by_an_emp,rating_duration,rating_variance,comp_azalutpt,comp_bhqczwkj,comp_bnivzbfi,comp_bucyzegb,comp_dmgwoqhz,comp_ejeyobsm,comp_ewpvmfbc,comp_fjslutlg,comp_fqsozvpv,comp_iqdwmigj,comp_jblrepyr,comp_jnvpfmup,comp_lgqwnfsg,comp_lydqevjo,comp_nmxkgvmi,comp_ocsicwng,comp_oecfwdaq,comp_ojidyfnn,comp_oqvaqcak,comp_pfmjacpm,comp_phcvroct,comp_pkeebtfe,comp_rcwkfavv,comp_rcyiinms,comp_rujnkvse,comp_siexkzzo,comp_spfcrgea,comp_ugldwwzf,comp_ujplihug,comp_vcqsbirc,comp_vwcdylha,comp_wsmblohy,comp_xccmgbjz,comp_ydqdpmvi,comp_ylpksopb,comp_yodaczsb,comp_zptfoxyq
0,0.488323,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.591810,-1.000000,-1.000000,1,0.000000,-1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.285714,3.288356,3.210502,3.202283,3.755479,3.172146,3.068493,3.043836,2.782877,2.199543,2.979680,2.974201,2.963242,2.957763,2.955023,2.946804,2.136721,1.903090,1.954243,2.298853,2.553883,2.103804,2.475671,2.774517,2.307496,2.089905,2.060698,2.418301,2.338456,1.826075,2.926857,3.0,5.0,7.0,10.0,16.0,7.0,11.0,19.0,6.0,12.0,6.0,11.0,3.0,5.0,12.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,1.0,3.0,-7.0,5.0,7.0,5.0,6.0,7.0,11.0,-1.0,-4.0,12.0,6.0,11.0,3.0,0.0,-3.0,5.0,5.0,7.0,11.0,18.0,7.0,11.0,23.0,8.0,12.0,6.0,11.0,3.0,6.0,15.0,0.230769,1.0,1.000000,0.666667,0.615385,1.0,1.0,0.487179,0.375,1.0,1.0,1.0,1.0,0.5,0.444444,3.137309,0.220365,0.573136,0.795782,1.963075,3.028743,0.889313,3.938217,112,1.895890,0.501927,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.062992,0.300000,2.524658,2.602511,2.683105,2.471461,2.310274,2.524658,2.602511,2.683105,2.471461,2.310274,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.431364,1.826075,1.698970,1.643453,1.612784,1.431364,1.826075,1.698970,1.643453,1.612784,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,4.0,11.0,0.0,4.0,4.0,4.0,11.0,0.0,4.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,11.0,-10.0,4.0,4.0,4.0,11.0,-10.0,4.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,11.0,2.0,4.0,4.0,4.0,11.0,2.0,4.0,4.0,-1.0,-1.0,-1.0,-

In [31]:
# function that returns the index of the position in the qlist which has the given value
def all_indices(value, qlist):
    indices = []
    idx = -1
    while True:
        try:
            idx = qlist.index(value, idx+1)
            indices.append(idx)
        except ValueError:
            break
    return indices

In [32]:
learning_rate_num = 1

# Use random forest with adaboost
classifier = RandomForestClassifier(criterion='entropy',max_depth=4,class_weight={0:1,1:5})
adaboost = AdaBoostClassifier(base_estimator=classifier,n_estimators=10,learning_rate=learning_rate_num)

# Split the train dataset into training and validation
X_train, X_test, y_train, y_test = train_test_split(train_final_df, target, test_size=0.2)

boostmodel = adaboost.fit(X_train, y_train)
y_pred = boostmodel.predict(X_test)

# Calculate the accuracy of Ones
indexes = all_indices(1,list(y_test))
predicted_ones = np.array(y_pred)[indexes]
only_ones_real = np.array(y_test)[indexes]
ones_accuracy = metrics.accuracy_score(only_ones_real, predicted_ones)

print("Accuracy ones:",ones_accuracy)

# Calculate the accuracy of zeroes
indexes = all_indices(0,list(y_test))
predicted_zeros = np.array(y_pred)[indexes]
only_zeros_real = np.array(y_test)[indexes]
zeros_accuracy = metrics.accuracy_score(only_zeros_real, predicted_zeros)

print("Accuracy zeros:",zeros_accuracy)

# give more weightabe to the accuracy of leaving the company
accuracy = (ones_accuracy * 5 + zeros_accuracy)/6

err = 0
z_count=0
p_count=0
result = list(y_test)

# calculate the Score 
for i in range(len(result)):
    if(result[i]==0):
        z_count += 1
        if(y_pred[i]==0):
            err += 1
    else:
        p_count += 1
        if(y_pred[i]==1):
            err += 5
print('score')
print(err/(z_count+(5*p_count)))

err = 0
result = y_test-y_pred
# Calculate the accuracy
for i in result:
    if i!=0:
        err += 1
print('accuracy')
print(1-err/len(result))

Accuracy ones: 0.8166666666666667
Accuracy zeros: 0.9453924914675768
score
0.8802698145025295
accuracy
0.9235127478753541


In [33]:
#### Train on the entire data and then predict on the test dataset
classifier = RandomForestClassifier(criterion='entropy',max_depth=4,class_weight={0:1,1:5})
adaboost = AdaBoostClassifier(base_estimator=classifier,n_estimators=10,learning_rate=learning_rate_num)
boostmodel = adaboost.fit(train_final_df, target)
left = boostmodel.predict(test_final_df )
ids = list(id_df)
dic ={'id': ids, 'left': left}
submission =  pd.DataFrame(dic) 
submission.to_csv('submissions.csv', index = False)

In [34]:
# Let us see which features contributed the most to the model.
zipped= zip(train_final_df.columns,boostmodel.feature_importances_)
res = sorted(zipped, key = lambda x: x[1]) 

In [35]:
res

[('comp_zptfoxyq', 0.0),
 ('comp_pkeebtfe', 2.669807365974447e-19),
 ('comp_jnvpfmup', 7.237494249830332e-18),
 ('comp_fjslutlg', 4.5300327618605763e-07),
 ('comp_bucyzegb', 4.939206419061248e-06),
 ('comp_oqvaqcak', 7.0699513305874744e-06),
 ('comp_rcyiinms', 1.0078888001253903e-05),
 ('comp_pfmjacpm', 1.3864305984603113e-05),
 ('comp_azalutpt', 1.9009392665303854e-05),
 ('comp_fqsozvpv', 2.099383803215457e-05),
 ('comp_nmxkgvmi', 4.616560497532256e-05),
 ('comp_lydqevjo', 8.447344603041223e-05),
 ('comp_dmgwoqhz', 9.589334139771019e-05),
 ('comp_rujnkvse', 0.000378245233231763),
 ('comp_ocsicwng', 0.00047203135059010435),
 ('comp_ugldwwzf', 0.000538545968263341),
 ('comp_ydqdpmvi', 0.0005956642541515931),
 ('oppose8', 0.0007232725413847875),
 ('oppose11', 0.0007720528628720417),
 ('comp_oecfwdaq', 0.0009087246442753549),
 ('oppose3', 0.0009519415310430201),
 ('comp_vcqsbirc', 0.0010258718776858821),
 ('comp_bnivzbfi', 0.0010563662024173164),
 ('comp_ujplihug', 0.001060576002474341),
